In [1]:
import cudf
import dask_cudf

In [1]:
import dask_cudf

In [3]:
from dask_cuml.linear_regression import LinearRegression

In order to run this notebook, you will first need to run a dask scheduler and number of dask workers:
- Run a dask scheduler with:  ```dask-scheduler --scheduler-file=cluster.json```
- Run N dask workers with:  ```mpirun -np N dask-mpi --no-nanny --nthreads 10 --no-scheduler --scheduler-file cluster.json```

In [4]:
# from dask.distributed import Client
# from dask_cuda import LocalCUDACluster
# cluster = LocalCUDACluster(threads_per_worker = 10)

In [5]:
from dask.distributed import Client
client = Client(scheduler_file = "../cluster.json")
client

Client Scheduler: tcp://10.2.169.115:8786 Dashboard: http://10.2.169.115:8787/status,Cluster Workers: 2 Cores: 2 Memory: 3.15 GB


In [6]:
import numba.cuda

devs = [i.id for i in numba.cuda.cudadrv.devices.gpus]
workers = list(client.has_what().keys())
worker_devs = workers[0:min(len(devs), len(workers))]

In [7]:
def set_visible(i, n):
    import os, numba.cuda
    all_devices = list(range(n))
    vd = ",".join(map(str, all_devices[i:] + all_devices[:i]))
    print(str(vd))
#     numba.cuda.close()
#     numba.cuda.select_device(i)
    print("Selecting Device : "  + str(i))
    os.environ["CUDA_VISIBLE_DEVICES"] = vd

dev_assigned = [client.submit(set_visible, dev, len(devs), workers = [worker]) for dev, worker in zip(devs, worker_devs)]

In [6]:
import pandas as pd

X = cudf.DataFrame([('a', [0, 1, 2, 3, 4])])
y = cudf.Series([0, 1, 2, 3, 4])


In [7]:
X_df = dask_cudf.from_cudf(X, chunksize=1).persist()
y_df = dask_cudf.from_cudf(y, chunksize=1).persist()

In [8]:
import numba.cuda
import cuml
def print_device(arr):
    print(str(numba.cuda.get_current_device()))
    print(str(cuml.device_of_ptr(arr.as_gpu_matrix(order="F"))))
    
[client.submit(print_device, part) for part in X_df.to_delayed()]

[<Future: status: pending, key: print_device-c34f15e6fd3fc6f7d125ba3b1a3e363b>,
 <Future: status: pending, key: print_device-c584118bee36d9d292c441f4b4c25fb6>,
 <Future: status: pending, key: print_device-ebd028d04b13df478cf8ebc0f312e06c>,
 <Future: status: pending, key: print_device-27656b9731ade4c15c6dbcb9e9a05323>]

In [9]:
client.who_has()

{"('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 0)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 1)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 2)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 3)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-ac3ac804029d47f5a38602d3af76b3a1', 0)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-ac3ac804029d47f5a38602d3af76b3a1', 1)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-ac3ac804029d47f5a38602d3af76b3a1', 2)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-ac3ac804029d47f5a38602d3af76b3a1', 3)": ('tcp://10.2.169.115:35905',),
 'print_device-27656b9731ade4c15c6dbcb9e9a05323': ('tcp://10.2.169.115:35905',),
 'print_device-c34f15e6fd3fc6f7d125ba3b1a3e363b': ('tcp://10.2.169.115:35895',),
 'print_device-c584118bee36d9d292c441f4b4c25fb6': ('tcp://10.2.169.115:35905',),
 'print_device-ebd028d04b13df478cf8ebc0f312e06c': ('tcp://10.2.169.115:35895'

Set each worker to host dfs on a different device. 

__Note__: You can ignore this if you started your workers with "CUDA_VISIBLE_DEVICE" already

In [10]:
client.who_has()

{"('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 0)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 1)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 2)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 3)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-ac3ac804029d47f5a38602d3af76b3a1', 0)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-ac3ac804029d47f5a38602d3af76b3a1', 1)": ('tcp://10.2.169.115:35905',),
 "('from_cudf-ac3ac804029d47f5a38602d3af76b3a1', 2)": ('tcp://10.2.169.115:35895',),
 "('from_cudf-ac3ac804029d47f5a38602d3af76b3a1', 3)": ('tcp://10.2.169.115:35905',),
 'print_device-27656b9731ade4c15c6dbcb9e9a05323': ('tcp://10.2.169.115:35905',),
 'print_device-c34f15e6fd3fc6f7d125ba3b1a3e363b': ('tcp://10.2.169.115:35895',),
 'print_device-c584118bee36d9d292c441f4b4c25fb6': ('tcp://10.2.169.115:35905',),
 'print_device-ebd028d04b13df478cf8ebc0f312e06c': ('tcp://10.2.169.115:35895'

In [36]:
lr = LinearRegression()

In [37]:
res = lr.fit(X_df, y_df)

input_devarrays: [(('10.2.169.115', 35905), <Future: status: finished, type: tuple, key: inputs_to_device_arrays-88e71bd4dab6703c6f780baf2246f006>), (('10.2.169.115', 35895), <Future: status: finished, type: tuple, key: inputs_to_device_arrays-1a7130e6bad11794760470e944f840b8>)]
exec_node: ('10.2.169.115', 35905)
ipc_handles: [<Future: status: pending, key: get_input_ipc_handles-e35559334da11da3761684ae75174a27>]
raw_arrays: [<Future: status: finished, type: tuple, key: inputs_to_device_arrays-88e71bd4dab6703c6f780baf2246f006>]


res

In [38]:
lr.coeffs

(('10.2.169.115', 35905),
 <Future: status: finished, type: DataFrame, key: get_result-ca14e26343e472a5e7e23faf971f9616>)

In [39]:
g = lr.predict(X_df)

WORKER PARTS: [(('10.2.169.115', 35895), <Future: status: finished, type: DataFrame, key: ('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 2)>), (('10.2.169.115', 35895), <Future: status: finished, type: DataFrame, key: ('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 0)>), (('10.2.169.115', 35905), <Future: status: finished, type: DataFrame, key: ('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 3)>), (('10.2.169.115', 35905), <Future: status: finished, type: DataFrame, key: ('from_cudf-2c6f7a8407df4197b7e7089ef5437d50', 1)>)]
ON WORKER: 2
NOT ON WORKER: 2
IPCHANDLES = [<Future: status: pending, key: get_ipc_handles-ef582c8cab4ddab14801ceae1ee3bdb2>, <Future: status: pending, key: get_ipc_handles-66a8f423966f01f911e2c1b70cd960a9>]
RAW_ARRAYS=[<Future: status: pending, key: as_gpu_matrix-c0bd179ad91a32029360d71a5fd01940>, <Future: status: pending, key: as_gpu_matrix-398e5b3890e9b428592dad5e4ec64a3d>]
f=<Future: status: finished, type: tuple, key: _predict_on_worker-1a85ea1c3fae3d8a84cd5c8bb68

In [40]:
g

tornado.gen.Return(<Future: status: finished, type: DataFrame, key: get_result-bad65ddc85a3d7da59504eb5fbeaf837>)